# IG04 - Pagos a Proveedores distintos a los definidos

El indicador IG11 verifica que en un documento de compra no se haya cambiado el proveedor con posterioridad. El riesgo que busca mitigar es el pago de bienes o servicios a proveedores erróneos y el área responsable es la Gerencia de Administración y Finanzas.

El siguiente código es para ejecutar el indicador de Auditoría Continua IG04 - Pagos a Proveedores distintos a los definidos.

 - ***Fecha creación: 18.08.2020*** 
 - ***Fecha modificación_1: 21.08.2020*** 
 - ***Fecha modificación_2: 03.09.2020*** 
 - ***Fecha modificación_3: 09.09.2020*** 
 - ***Fecha modificación_4: 06.10.2020*** 
 - ***Fecha modificación_5: 13.11.2020*** 

## Importar librerías
---

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import string

## Utilidades

In [2]:
def clean_blankspace(df):
    # Eliminar espacios en nombres de columnas
    columns = list(df.columns)
    columns = [w.replace(' ','') for w in columns]
    df.columns = columns
    
    # Eliminar espacios en cada celda del dataframe
    for columna in list(df.columns):
        if df.dtypes[columna] == np.dtype('O'):
            df[columna] = df[columna].str.replace(' ','')
    return df

def clean_columns(txt):
    df = txt.drop(columns=txt.columns[0:2]).drop(columns=txt.columns[-1]).dropna(how='all').reset_index(drop=True)
    return df

def num_format(df,col):
    df[col] = df[col].str.replace('.','').str.replace(',','.').astype(float)
    if (col=='DMBTR') or (col=='ERFMG') or (col=='VERPR'):
        df[col] = df[col]*100
    return df

def conversion_clp(waers,rlwrt):
    if waers == 'CLP':
        valor = rlwrt*100
    if waers == 'EUR':
        valor = rlwrt*906
    if waers == 'GBP':
        valor = rlwrt*1004.62
    if waers == 'UF':
        valor = (rlwrt/1000)*28915
    if waers == 'USD':
        valor = rlwrt*766
    if waers == 'UTM':                 
        valor = (rlwrt/100)*50674
    if waers == 'AUD':   
        valor = rlwrt*500.11
    if waers == 'CAD':
        valor = rlwrt*600.8
    if (waers == '') or (waers == 'nan'):
        valor = rlwrt
    return valor

def str_format(df,col):
    
    #Elimina letras, sinbolos de puntuacion, u otros simbolos de columnas numericas
    
    punctuation = list(string.punctuation)   # lista de simbolos de puntuacion      
    letters = list(string.ascii_letters)     # Lista de letras mayusculas y minusculas        
    symbols = ['^','°']                      # Lista de simbolos. Si aparece uno nuevo, habría que agregarlo a la lista!

    for i in range(len(punctuation)):
        df[col] = df[col].str.replace(punctuation[i],'')   # Elimina letras

    for i in range(len(letters)):
        df[col] = df[col].str.replace(letters[i],'')       # Elimina simbolos de puntuación
    
    for i in range(len(symbols)):
        df[col] = df[col].str.replace(symbols[i],'')       # Elimina otros simbolos

    df = df[df[col] != '']                                 # Elimina celdas sin contenido
    
    return df

def val(row):
#validar que los proveedores de la tabla ekko no cambiaron con posterioridad para un mismo documento de compra y material
    if row['LIFNR_ekko'] == row['LIFNR']:
        val = 'Igual'
    else:
        val = 'Distinto'
    return val

## Cargar tablas
---

In [3]:
ekko_df = pd.read_csv('EKKO.txt', sep='|',header=3, encoding='latin1', low_memory=False)
ekbe_df = pd.read_csv('EKBE.txt', sep='|',header=3, encoding='latin1', low_memory=False)
rbkp_df = pd.read_csv('RBKP.txt', sep='|',header=3, encoding='latin1', low_memory=False)
lfa1_df = pd.read_csv('LFA1.txt', sep='|',header=3, encoding='latin1', low_memory=False)

## Limpieza de tablas
---

**Eliminar columnas y filas nulas**

In [4]:
ekko_df = clean_columns(ekko_df)
ekbe_df = clean_columns(ekbe_df)
rbkp_df = clean_columns(rbkp_df)
lfa1_df = clean_columns(lfa1_df)

#### Eliminar espacios en columnas y cada celda

In [5]:
ekko_df = clean_blankspace(ekko_df)
ekbe_df = clean_blankspace(ekbe_df)
rbkp_df = clean_blankspace(rbkp_df)
lfa1_df = clean_blankspace(lfa1_df)

#### Cambio de formato de columnas

In [6]:
ekko_df = num_format(ekko_df,'RLWRT')                              # reemplaza "," por "", y "," por "."
ekko_df = ekko_df.replace(r'^\s*$', np.nan, regex=True)            # reemplaza valores vacios con np.nan

ekbe_df = str_format(ekbe_df,'EBELN')                              # elimina letras o simbolos de columnas numericas  

rbkp_df = str_format(rbkp_df,'XBLNR')                              # elimina letras o simbolos de columnas numericas  
rbkp_df = num_format(rbkp_df,'RMWWR')                              # reemplaza "," por "", y "," por "."
rbkp_df = rbkp_df.replace(r'^\s*$', np.nan, regex=True)            # reemplaza valores vacios con np.nan

#### Establecer esquema de datos

In [7]:
schema_ekko = {'EBELN': 'float64',     # Doc_compra 
                'BUKRS': str,          # Sociedad 
                'BSART': str,          # Clase_doc_compra 
                'AEDAT': str,          # Fecha_creacion 
                'LIFNR': 'float64',    # Proveedor_ekko 
                'WAERS': str,          # Moneda
                'FRGKE': str,          # Ind. Liberación 
                'PROCSTAT': 'float64', # Estatus_doc 
                'RLWRT': 'float64'}    # Monto_neto

schema_ekbe = {'EBELN': 'float64',     # Doc_compra 
                'EBELP': 'float64',    # Posicion 
                'BELNR': 'float64'}    # Doc_material

schema_rbkp = {'BELNR': 'float64',     # Doc_material 
                'GJAHR': 'float64',    # Año 
                'BLART': str,          # Clase_doc_material 
                'BUDAT': str,          # Fecha_contabilidad 
                'USNAM': str,          # Usuario 
                'XBLNR': 'float64',    # Referencia 
                'LIFNR': 'float64',    # Proveedor 
                'WAERS': str,          # Moneda 
                'RMWWR': 'float64'}    # Monto_bruto

schema_lfa1 = {'LIFNR': 'float64',     # Proveedor
                'NAME1': str,          # Nombre_proveedor
                'SORTL': str,          # Rut_proveedor
                'BRSCH': str,          # Tipo_proveedor
                'KTOKK': str}          # Grupo_de_cuentas

ekko_df = ekko_df.astype(schema_ekko)
ekbe_df = ekbe_df.astype(schema_ekbe)
rbkp_df = rbkp_df.astype(schema_rbkp)
lfa1_df = lfa1_df.astype(schema_lfa1)

**Aplicar filtros que se utilizan para este indicador**

In [8]:
# Filtro por clase de documento (documentos a proveedor)

ekko_df = ekko_df[(ekko_df['BSART']=='ZMAT') | (ekko_df['BSART']=='ZSER') | (ekko_df['BSART']=='ZSCL') | (ekko_df['BSART']=='ZCDP') | (ekko_df['BSART']=='ZODA')]

#### Cambio de formato de columnas en su respectiva moneda

In [10]:
ekko_df['RLWRT_CLP'] = ekko_df.apply(lambda x: conversion_clp(x['WAERS'],x['RLWRT']),axis=1)
rbkp_df['RMWWR_CLP'] = rbkp_df.apply(lambda x: conversion_clp(x['WAERS'],x['RMWWR']),axis=1)

**Eliminar columnas que no se utilizan en este indicador**

In [11]:
ekko_df.drop(columns= ['PROCSTAT','WAERS','RLWRT','FRGKE'], inplace= True)
rbkp_df.drop(columns= ['RMWWR'], inplace= True)

#### Cambio de nombre de columnas (para diferenciar en cruce)

In [12]:
ekko_df.rename(columns={'LIFNR':'LIFNR_ekko'}, inplace= True)

## Ejecución indicador
---

Primero, hacemos el siguente cruce:
 - ekko y ekbe: utilizando como atributo clave el campo `Doc_compra`
 - rbkp y lfa1: utilizando como atributo clave el campo `Proveedor`

**ekko y ekbe**

In [13]:
ekko_ekbe = pd.merge(ekko_df, ekbe_df, on='EBELN', how='inner')
ekko_ekbe = ekko_ekbe.drop_duplicates()

**rbkp y lfa1**

In [14]:
rbkp_lfa1 = pd.merge(rbkp_df, lfa1_df, on='LIFNR', how='inner')
rbkp_lfa1 = rbkp_lfa1.drop_duplicates()

**Luego, cruzamos ambas tablas (ekko_ekbe y rbkp_lfa1) por `Doc_material`**

In [15]:
IG11 = pd.merge(ekko_ekbe, rbkp_lfa1, on='BELNR', how='inner')
IG11 = IG11.drop_duplicates()

##### Reordenamiento de columnas

In [16]:
# Ordenamos las columnas para mejor entendimiento

IG11 = IG11[['BUKRS','BSART','EBELN','EBELP','LIFNR_ekko','AEDAT','RLWRT_CLP','BELNR','LIFNR','NAME1','BUDAT','USNAM','RMWWR_CLP','WAERS']]

Funcion val() para validar que los proveedores de la tabla ekko no cambiaron con posterioridad para un mismo documento de compra y material. Aplicamos la función sobre la tabla con todos los cruces

In [17]:
IG11['Validacion'] = IG11.apply(val, axis=1)

Por último, extraemos los valores que tengan **"Distinto"** en la columna Validación 

In [18]:
IG11 = IG11.loc[(IG11['Validacion']=='Distinto'),:]

#### Cambio de nombre de columnas

In [19]:
IG11.rename(columns={'BUKRS':'Sociedad',
                          'EBELN':'Orden_Compra',
                          'BSART':'Clase_doc_compra',
                          'BELNR':'Factura',
                          'BLART':'Clase_doc_material',
                          'LIFNR':'Proveedor_Factura',
                          'NAME1':'Nombre_proveedor',
                          'GJAHR':'Año',
                          'AEDAT':'Fecha_creacion_OC',
                          'BUDAT':'Fecha_cont_Fact',
                          'EBELP':'Posicion',
                          'USNAM':'Usuario',
                          'RLWRT_CLP':'Monto_Total_OC',
                          'RMWWR_CLP':'Monto_Total_Fact',
                          'WAERS':'Moneda',
                          'LIFNR_ekko':'Proveedor_OC',
                          'Validacion':'Validacion'}, 
            inplace = True)

## Guardar en Excel

In [20]:
nombre_archivo = 'IG11 '+datetime.now().strftime("%d-%m-%y_%Hh%Mm")+'.xlsx'
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')

IG11.to_excel(writer, sheet_name='IG11')

writer.save()